# Forecasting deposit stickyness in custodian banking institutions  

## Project Description: 
I currently work for a large custody bank that as most other organizations are dealing with uncertainty driven by the Covid -19 epidemic.  Unlike many other retail-oriented services we experience deposit growth during periods of stress.  The question that arises in all of our executive team meetings is if the deposits will stay on, and if not how long will the duration of elevated deposits be. The importance to my field of work is determining how will those coming deposit actions of our clients will influence the liquidity position of the bank, and how we need to manage preemptively for those impending actions. This project leveraging historical balance sheet information available from custodian banking institutions, will forecast ramp up and ramp down inflection points for deposit growth and subsequent decline using various macroeconomic indicators as independent variables in an effort to get a sense of what’s to come for our bank's deposit behavior. 

## Data Science Task: 
To determine and apply the most appropriate supervised learning algorithm to forecast deposit inflection points during economic stress, in an effort to forecast current deposit trends at my banking institution

## Data:  
This project will take historical balance sheet information, specifically foreign office, retail, and commercial deposits from some of the large custody banks including Northern Trust, Bank of New York, and State Street Bank & Trust using publicly available balance sheet information from SNL or via the Edgar API tool, and then obtain historical macroeconomic indicators from FRED, including interest rates, stock indices like Dow Jones S&P, GDP Growth, CPI, VIX unemployment and treasury yield in an effort to forecast the inflection point in deposit growth and subsequent declines resulting from periods of economic stress. 
For this effort I will be looking at quarterly banking deposit data as my dependent variable, I’m going to try and go back as far as possible with historical data hoping to capture at the least the 1987 black Monday related recession, this data will be obtained via S&P’s SNL and or the EDGAR API [2].  For my independent variables I plan to leveraging FRED pretty heavily to obtain the most up to date historical macroeconomic indicator information, and may grab data via the FRED API [3]. 
In terms of data size, while there will be many variables to start with, this will be unlike a twitter stream project that will collect immense amounts of information, this project is finite and can potentially only go back to the 1980’s so data size and processing time due to quantity is not of concern.  

## Data Analysis:
I plan to take Jason Brownlee’s approach of spot-checking various supervised learning algorithms including potentially regression, support vector machines, random forest, LDA, naïve Bayes, decision tree, KNN and ANN to determine the best fit to predict deposit inflection points [1]. I obviously have many macro-economic indicators and will do a clean up to find the most impactful indicators to explain and forecast deposit movement, in an effort to understand indicators that impact this behavior the most, and use that to predict future deposit behavior

## First steps - obtain dependent variable data via FDIC API: 
The goal is to obtain deposit information as our dependent variable.  For the sake of consistency I'll be looking at three custodian banking institutions to see if I can observe consistent trends. The stock tickers of the banks are Northern Trust (NTRS), Bank of NY Mellon (BONY), and State Street Bank & Trust (SST).

First we will install the swagger client API to pull historical call data from the FDIC [4]: 

In [74]:
pip install bankfind

Next we import the client

In [75]:
import bankfind as bf
# Get historical data
data = bf.get_summary(filters="NAME:Northern Trust")
data                    

{'meta': {'total': 0,
  'parameters': {'filters': 'NAME:Northern Trust',
   'fields': 'ALLOTHER,alsonew,ASSET,BANKS,BKPREM,BRANCHES,BRANCHIN,BRO,BRWDMONY,CB_SI,chartoth,CHBAL,CHBALI,chrtrest,comboass,combos,CONS,CORPBNDS,COUNT,CRLNLS,DDT,DEP,DEPDOM,DEPFOR,DEPI,DEPIFOR,DEPNI,DEPNIFOR,DRLNLS,EAMINTAN,EDEP,EDEPDOM,EDEPFOR,EEREPP,EFHLBADV,EFREPP,EINTEXP,EINTEXP2,ELNATR,EOTHNINT,EPREMAGG,EQ,EQCDIV,EQCDIVC,EQCDIVP,EQCS,EQDIV,EQNM,EQNWCERT,EQOTHCC,EQPP,EQSUR,EQUPTOT,ESAL,ESUBND,EXTRA,FD_BIF,FD_SAIF,FREPO,FREPP,ICHBAL,IFEE,IFREPO,IGLSEC,ILNDOM,ILNFOR,ILNLS,ILNS,ILS,INTAN,INTBAST,INTBLIB,INTINC,INTINC2,IRAKEOGH,ISC,ISERCHG,ITAX,ITAXR,ITRADE,LIAB,LIABEQ,liqasstd,liqunass,LNAG,LNALLOTH,LNATRES,LNAUTO,LNCI,LNCON,LNCONOT1,LNCONOTH,LNCRCD,LNDEP,LNLS,LNLSGR,LNLSNET,LNMOBILE,LNMUNI,LNRE,LNREAG,LNRECONS,LNREDOM,LNREFOR,LNRELOC,LNREMULT,LNRENRES,LNRERES,LNRESRE,LNSP,LS,MERGERS,MISSADJ,MTGLS,NALNLS,NCHGREC,NCLNLS,NETINC,newcount,New_Char,NEW6_1,NEW9_1,NEW10_1,NEW10_2,NEW10_3,NEW11_1,NEW14_1,NEW14_2,NEW14

With a little bit of digging around the FDIC has the data I need but not for bulk download via their website.  I will move forward to trying to obtain the data using the EDGAR API 

In [76]:
# package used to execute HTTP POST request to the API
import json
import urllib.request

# API Key
TOKEN = 'c2a2fd4e216fa2421707f68cb6a95a79536a04c4fa258cdcbd9a0a8d09379dcb'
# API endpoint
API = "https://api.sec-api.io?token=" + TOKEN

# define the filter parameters you want to send to the API 
payload = {
  "query": { "query_string": { "query": "cik:73124 AND filedAt:{2001-01-01 TO 2020-09-30} AND formType:\"10-\"" } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

# format your payload to JSON bytes
jsondata = json.dumps(payload)
jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes

# instantiate the request 
req = urllib.request.Request(API)

# set the correct HTTP header: Content-Type = application/json
req.add_header('Content-Type', 'application/json; charset=utf-8')
# set the correct length of your request
req.add_header('Content-Length', len(jsondataasbytes))

# send the request to the API
response = urllib.request.urlopen(req, jsondataasbytes)

# read the response 
res_body = response.read()
# transform the response into JSON
filings = json.loads(res_body.decode("utf-8"))

# print JSON 
print(filings)

{'total': {'value': 71, 'relation': 'eq'}, 'query': {'from': 0, 'size': '10'}, 'filings': [{'ticker': 'NTRS', 'formType': '10-Q', 'accessionNo': '0000073124-20-000212', 'cik': '73124', 'companyNameLong': 'NORTHERN TRUST CORP (Filer)', 'companyName': 'NORTHERN TRUST CORP', 'linkToFilingDetails': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/73124/000007312420000212/ntrs-20200630.htm', 'description': 'Form 10-Q - Quarterly report [Sections 13 or 15(d)]', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/73124/000007312420000212/0000073124-20-000212.txt', 'filedAt': '2020-07-28T16:48:23-04:00', 'documentFormatFiles': [{'sequence': '1', 'size': '8324715', 'documentUrl': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/73124/000007312420000212/ntrs-20200630.htm', 'description': '10-Q', 'type': '10-Q'}, {'sequence': '2', 'size': '13171', 'documentUrl': 'https://www.sec.gov/Archives/edgar/data/73124/000007312420000212/q22020ex311.htm', 'description': 'EX-31.1', 'type': 'EX-31.1'}, {'se

In [77]:
json.dumps(filings, indent=4)

'{\n    "total": {\n        "value": 71,\n        "relation": "eq"\n    },\n    "query": {\n        "from": 0,\n        "size": "10"\n    },\n    "filings": [\n        {\n            "ticker": "NTRS",\n            "formType": "10-Q",\n            "accessionNo": "0000073124-20-000212",\n            "cik": "73124",\n            "companyNameLong": "NORTHERN TRUST CORP (Filer)",\n            "companyName": "NORTHERN TRUST CORP",\n            "linkToFilingDetails": "https://www.sec.gov/ix?doc=/Archives/edgar/data/73124/000007312420000212/ntrs-20200630.htm",\n            "description": "Form 10-Q - Quarterly report [Sections 13 or 15(d)]",\n            "linkToTxt": "https://www.sec.gov/Archives/edgar/data/73124/000007312420000212/0000073124-20-000212.txt",\n            "filedAt": "2020-07-28T16:48:23-04:00",\n            "documentFormatFiles": [\n                {\n                    "sequence": "1",\n                    "size": "8324715",\n                    "documentUrl": "https://www.se

Not quite what I had anticipated, I will try another method on the EDGAR API links 

## Independent Data Step: 
We will obtain all the necesary data for our independent variables, whose sources are largely FRED and Yahoo Finance for the Dow Jones and S&P 500 data [6][7]

In [80]:
#Vix 
import datetime
vix = pd.read_csv('VIXCLS.csv')
vix['DATE'] = pd.to_datetime(vix['DATE'])

In [81]:
#Fed Funds Rate
ff = pd.read_csv('FEDFUNDS (1).csv')
ff['DATE'] = pd.to_datetime(ff['DATE'])

In [82]:
#Start Data Frame with all independent variables 
import numpy as np
indep = pd.merge(ff,vix, 
                how = 'right')
indep = indep.fillna(value=np.nan)

In [83]:
#CPI
cpi = pd.read_csv('CPIAUCSL.csv')
cpi['DATE'] = pd.to_datetime(cpi['DATE'])
#merge with indep
indep = pd.merge(indep,cpi, 
                how = 'right')

In [86]:
#Dow Jones Industrial Average
djii = pd.read_csv('HistoricalPrices.csv')
djii['Date'] = pd.to_datetime(djii['Date'])
djii['Date'] = djii.rename(columns = {'DATE'})
#merge with indep
indep = pd.merge(indep,djii, 
                how = 'right')

TypeError: 'set' object is not callable

In [79]:
#Clean up indep - remove open low and high from djii & change column name to DJI Close 
indep = indep.drop(['Open', 'High','Low'], axis = 1, inplace = True)

KeyError: "['Open' 'High' 'Low'] not found in axis"

### References
[1] python-edgar. (2019, November 9). Retrieved from https://pypi.org/project/python-edgar/
[2] Schroeder, J. (2019, August 27). Tutorial: real-time live feed of SEC filings using Python & socket.io. Retrieved from https://medium.com/@jan_5421/crawling-new-filings-on-sec-edgar-using-python-and-socket-io-in-real-time-5cba8c6a3eb8
[3] Chen, K. (2020). Use Python to download TXT-format SEC filings on EDGAR (Part I) | Kai Chen. Retrieved October 29, 2020, from http://kaichen.work/?p=59
[4] A. (2020). amaline/fdic-banks-api-python-client. Retrieved from https://github.com/amaline/fdic-banks-api-python-client
[5] D. (2020b). dpguthrie/bankfind. Retrieved from https://github.com/dpguthrie/bankfind
[6] Federal Reserve Economic Data | FRED | St. Louis Fed. (2020). Retrieved from https://fred.stlouisfed.org/
[7]Yahoo is now a part of Verizon Media. (2020). Retrieved from https://finance.yahoo.com/quote/%5EGSPC/history/